In [2]:
import os
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model


In [3]:
# Paths to your saved models
AGE_MODEL_PATH      = "child_detection.keras"
CRIMINAL_MODEL_PATH = "criminal_detection.keras"

# Load the models once at import
age_model      = load_model(AGE_MODEL_PATH)
criminal_model = load_model(CRIMINAL_MODEL_PATH)

# Criminal class names must match the training order
CRIMINAL_CLASSES = ['Srinivas']  # extend this list if you have more classes

# Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Image size used during training
IMG_SIZE = 200


In [4]:
def process_face_frame(frame):
    """
    Detects faces in `frame`, classifies each as child or criminal,
    draws colored boxes and labels, and returns:
      - annotated_frame: the frame with overlays
      - child_boxes: list of (x, y, w, h) for faces classified as children
    """
    annotated = frame.copy()
    gray = cv2.cvtColor(annotated, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    child_boxes = []

    for (x, y, w, h) in faces:
        roi = annotated[y:y+h, x:x+w]
        # Resize to the expected input shape for the age model (200x200)
        face = cv2.resize(roi, (200, 200)).astype("float32") / 255.0
        face_batch = np.expand_dims(face, axis=0)

        # 1) Age estimation
        age_prob = age_model.predict(face_batch, verbose=0)[0][0]
        is_child = (age_prob >= 0.1)

        if is_child:
            label = "Child"
            color = (0, 255, 0)  # Green
            child_boxes.append((x, y, w, h))
        else:
            # 2) Criminal detection
            crim_probs = criminal_model.predict(face_batch, verbose=0)[0]
            crim_idx   = np.argmax(crim_probs)
            crim_conf  = crim_probs[crim_idx]

            if crim_conf == 1.0:
                label = CRIMINAL_CLASSES[crim_idx]
                color = (0, 0, 255)  # Red
            else:
                continue  # Neither child nor known criminal

        # Draw rectangle and label
        cv2.rectangle(annotated, (x, y), (x + w, y + h), color, 2)
        cv2.putText(annotated, label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    return annotated, child_boxes


In [5]:
# Path to your trained activity model
ACTIVITY_MODEL_PATH = "activity_monitoring.keras"
activity_model = load_model(ACTIVITY_MODEL_PATH)

# Activity labels (order must match training)
CLASS_LABELS = ['CricketShot', 'PlayingCello', 'Punch', 'ShavingBeard', 'TennisSwing']

# Define normal and abnormal activities
NORMAL_ACTIVITIES = ['CricketShot', 'PlayingCello', 'TennisSwing']
ABNORMAL_ACTIVITIES = ['Punch', 'ShavingBeard']


def preprocess_video_frames(frames, target_size=(64, 64)):
    """
    Preprocesses the list of frames to make them compatible with the model.
    Resizes and normalizes the frames before feeding them into the model.
    """
    processed = []
    for frame in frames:
        resized = cv2.resize(frame, target_size)
        normalized = resized.astype("float32") / 255.0
        processed.append(normalized)
    return np.array(processed)


def predict_activity_from_frames(frames):
    """
    Predicts the activity based on the most recent frames.
    Returns:
      - activity label
      - status ('Normal' or 'Abnormal')
      - color (for display)
    """
    if len(frames) < 20:
        return "Unknown", "Insufficient frames", (0, 0, 255)

    sampled_frames = frames[-20:]  # Last 20 frames only
    processed = preprocess_video_frames(sampled_frames, target_size=(64, 64))
    input_data = np.expand_dims(processed, axis=0)  # Shape: (1, 20, 64, 64, 3)

    predictions = activity_model.predict(input_data, verbose=0)[0]
    pred_index = np.argmax(predictions)
    label = CLASS_LABELS[pred_index]

    if label in NORMAL_ACTIVITIES:
        return label, "Normal", (0, 255, 0)  # Green
    else:
        return label, "Abnormal", (0, 0, 255)  # Red


In [6]:
# Constants
IMG_SIZE = 224
MODEL_PATH = "behaviour_analysis.keras"

# Load the trained behavior model
behavior_model = load_model(MODEL_PATH)

def predict_behavior(frame):
    """
    Predicts if the behavior in a given frame is normal or abnormal.
    Returns the label and the color (green/red) for bounding box.
    """
    resized = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    normalized = resized.astype('float32') / 255.0
    img = np.expand_dims(normalized, axis=0)
    
    prediction = behavior_model.predict(img, verbose=0)[0][0]

    if prediction <= 0.9:
        return "Normal", (0, 255, 0)  # Green
    else:
        return "Abnormal", (0, 0, 255)  # Red

def analyze_behavior_on_frame(frame, detected_children):
    """
    Applies behavior analysis on all detected children in the frame.
    `detected_children` is a list of (x, y, w, h) for each child.
    """
    for (x, y, w, h) in detected_children:
        child_face = frame[y:y+h, x:x+w]

        if child_face.size == 0:
            continue

        label, color = predict_behavior(child_face)

        # Draw rectangle around the face (already drawn in previous step, optional here)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

        # Show "Normal" or "Abnormal" label INSIDE the box (bottom-left)
        text_position = (x + 5, y + h - 10)  # Slight padding from bottom-left
        cv2.putText(frame, label, text_position, cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, color, 2)

    return frame


In [7]:
def main():
    print("👋 Welcome to ChildGuard")
    print("Enter input mode: 'c' for Camera, 'v' for Video path")
    print("Press 'q' to quit anytime ❌")

    user_input = input("Your choice: ").strip().lower()

    if user_input == 'q':
        print("👋 Quitting the program. Stay safe!")
        return

    cap = None
    activity_frames = []

    if user_input == 'v':
        video_path = input("🎞️ Enter video file path: ").strip()

        # Sanitize path
        video_path = video_path.strip('"').strip("'").replace("\\", "/")
        print(f"🧾 Cleaned path: {video_path}")

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("⚠️ Error: Cannot open the video file. Switching to camera 🎥.")
            cap = cv2.VideoCapture(0)
        else:
            print("Video loaded successfully! 🎬")
    else:
        cap = cv2.VideoCapture(0)
        print("📸 Starting camera feed...")

    if not cap.isOpened():
        print("❌ Error: Could not access camera or video.")
        return

    print("🚀 Processing started. Press 'q' to quit early.")
    start_time = time.time()
    while True:
        ret, frame = cap.read()
        if not ret:
            print("📭 End of stream or can't read the frame.")
            break

        # Store frames for activity detection
        activity_frames.append(frame)

        # Process frame using face recognition (from ChildGuard)
        annotated_frame, child_boxes = process_face_frame(frame)

        # Process activity recognition from stored frames
        activity_label, activity_status, activity_color = predict_activity_from_frames(activity_frames)

        # Add activity label to the annotated frame
        cv2.putText(annotated_frame, f"Activity: {activity_label} ({activity_status})",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, activity_color, 2)

        # Apply behavior analysis
        final_frame = analyze_behavior_on_frame(annotated_frame, child_boxes)

        # Display the annotated frame
        cv2.imshow('ChildGuard - Press q to quit', final_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("👋 Quitting early by user. See you next time!")
            break

        if time.time() - start_time > 30:
            print("⏱️ 30 seconds completed. Shutting down gracefully 💤")
            break

    cap.release()
    cv2.destroyAllWindows()
    print("🧹 Resources cleaned up. Goodbye! 👋")



In [8]:
main()

👋 Welcome to ChildGuard
Enter input mode: 'c' for Camera, 'v' for Video path
Press 'q' to quit anytime ❌


Your choice:  v
🎞️ Enter video file path:  "E:\6. WIN SEM -- 2024-2025\F1. DL\Child Surveillance DL Project\2.mp4"


🧾 Cleaned path: E:/6. WIN SEM -- 2024-2025/F1. DL/Child Surveillance DL Project/2.mp4
Video loaded successfully! 🎬
🚀 Processing started. Press 'q' to quit early.
⏱️ 30 seconds completed. Shutting down gracefully 💤
🧹 Resources cleaned up. Goodbye! 👋
